In [112]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string

import tweepy                  #Getting twitter data like tweets, followers, friends

import nltk
from textblob import TextBlob  #Sentiment analysis

import networkx as nx          #Drawing & analyzing network

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [113]:

#--------------------------------------------------

t_credentials = dict()
#These are the credentials obtained by setting up your twitter developer account
t_credentials['CONSUMER_KEY'] = 'RmYMHj3RwVL2qj2sB9V0jbGB2'  
t_credentials['CONSUMER_SECRET'] = 'rwkLatJck2pvHEeljerwWfwYi6Wf0Tquy90UyERunzFeXCAlW7'
t_credentials['TOKEN_KEY'] = '590766984-MJRgfipJLzhQt2R6VcktfsANXtiH6Rs87MUpl6ly'
t_credentials['TOKEN_SECRET'] = 'P6hl0Lwfclbcz4JjAWb9ILrTkyrPGB6cpd0UVGKmgGcZU'

#load Twitter API credentials
consumer_key = t_credentials['CONSUMER_KEY']
consumer_secret = t_credentials['CONSUMER_SECRET']
access_key = t_credentials['TOKEN_KEY']
access_secret = t_credentials['TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)


#------------------------------------------



In [188]:
hash_tags = '#lebanonprotest-filter:retweets'
#tweet_df = pd.read_csv('100000_Tweets.csv',lineterminator='\n')
tweet_cols = ['created_at','screen_name','location','followers_count','friends_count','retweeted','retweet_count',
              'text','tags','mentions','source']
tweet_df['dummy_count'] = 1
tweet_df = pd.DataFrame(columns = tweet_cols)
for tweet in tweepy.Cursor(api.search,q=hash_tags, result_type='mixec',  #Example Values: mixed, recent, popular
                           lang="en",tweet_mode='extended',until='2019-10-20',wait_on_rate_limit=True).items(40000):
    tags=[]
    for i in range(len(tweet.entities['hashtags'])):
        tags.append('#'+tweet.entities['hashtags'][i]['text'].lower())
    mentions = []
    for i in range(len(tweet.entities['user_mentions'])):
        mentions.append('@'+tweet.entities['user_mentions'][i]['screen_name'])
    df = pd.DataFrame([[tweet.created_at,tweet.user.screen_name,tweet.user.location,tweet.user.followers_count,
                        tweet.user.friends_count,tweet.retweeted,tweet.retweet_count,tweet.full_text,tags,mentions,tweet.source]],columns = tweet_cols)
    tweet_df = tweet_df.append(df)
    tweet_df_rows = tweet_df.shape[0]
    if(tweet_df_rows%100==0):
        print(str(tweet_df_rows)+'---'+str(df['created_at']))
tweet_df.reset_index(drop=True,inplace=True)
tweet_df = tweet_df.sort_values(['created_at'],ascending=[False])

#Map the twitter user names into ids

tweeters = sorted(list(set(list(tweet_df['screen_name']))))
tweeters_id = [('user_'+str(i)) for i in range(len(tweeters))]
mapping_dict = {tweeters[i]:tweeters_id[i] for i in range(len(tweeters))}
tweet_df['screen_name'] = tweet_df['screen_name'].map(mapping_dict)


def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def fill_sentiments(tweet_df):
    tweet_df['tweet_sentiment_polarity'] = 0.0
    tweet_df['tweet_sentiment_subjectivity'] = 0.0
    for index,row in tweet_df.iterrows():
        cleaned_tweet = row['text']
        s_analysis = TextBlob(cleaned_tweet)
        tweet_df.at[index,'tweet_sentiment_polarity'] = s_analysis.sentiment.polarity
        tweet_df.at[index,'tweet_sentiment_subjectivity'] = s_analysis.sentiment.subjectivity
    return tweet_df

tweet_df = fill_sentiments(tweet_df)

print(tweet_df.shape)
print(tweet_df['created_at'].min())
print(tweet_df['created_at'].max())
print(tweet_df['retweet_count'].max())
print(tweet_df['mentions'].max())
#print (tweet_df['mentions'])

tweet_df.head()



100---0   2019-10-19 15:40:58
Name: created_at, dtype: datetime64[ns]
200---0   2019-10-19 12:03:08
Name: created_at, dtype: datetime64[ns]
300---0   2019-10-19 01:16:40
Name: created_at, dtype: datetime64[ns]
(347, 13)
2019-10-17 20:21:32
2019-10-19 23:39:12
840
['@walidjoumblatt']


,created_at,screen_name,location,followers_count,friends_count,retweeted,retweet_count,text,tags,mentions,source,tweet_sentiment_polarity,tweet_sentiment_subjectivity
0,2019-10-19 23:39:12,user_264,Stark State,6797,2709,False,0,"whoever is in Sydney and is Lebanese, I’ll be ...","[#lebanon, #lebanese, #lebanonprotest, #lebane...",[],Twitter for iPhone,0.000000,0.000000
1,2019-10-19 23:31:35,user_16,Los Angeles,104,195,False,0,@antoinesaad12 #Lebanon #lebanonprotest We th...,"[#lebanon, #lebanonprotest]",[@antoinesaad12],Twitter for iPhone,0.000000,0.000000
2,2019-10-19 23:16:46,user_151,"Melbourne, Victoria",285,186,False,2,Even during their time at night protesting aga...,"[#lebanon, #lebanonprotest, #lebanesetakeover,...",[],Twitter for iPhone,0.250000,0.350000
3,2019-10-19 23:07:14,user_193,Canada,352,254,False,2,@joemaalouftv Change can be at times difficult...,"[#lebanonprotest, #لبنان_ينتفض]",[@joemaalouftv],Twitter for iPad,0.257143,0.727143
4,2019-10-19 22:57:50,user_146,United States,13399,7846,False,0,"Thread by @love4thegameAK: ""Unbelievable prote...","[#lebanonprotest, #wwg1wga_worldwide, #lebanon]",[@love4thegameAK],Twitter for iPad,-0.256250,0.618750


In [115]:
len(set(tweet_df['screen_name']))

273

#### calculate sentiment in tweets

In [116]:
retweet_threshold=10
tweet_df['dummy_count'] = 1
retweets_df = tweet_df[['screen_name','retweet_count']].groupby('screen_name').sum()
followers_df = tweet_df[['screen_name','followers_count']].groupby('screen_name').max() #Max of the week
friends_df= tweet_df[['screen_name','friends_count']].groupby('screen_name').max()  #Max of the week
sentiment_polarity_df = tweet_df[['screen_name','tweet_sentiment_polarity']].groupby('screen_name').sum()
sentiment_subjectivity_df = tweet_df[['screen_name','tweet_sentiment_subjectivity']].groupby('screen_name').sum()
tweet_count_df = tweet_df[['screen_name','dummy_count']].groupby('screen_name').sum()

stats_df = pd.concat([tweet_count_df,retweets_df,followers_df,friends_df,
                            sentiment_polarity_df,sentiment_subjectivity_df],axis='columns')
stats_df['screen_name'] = stats_df.index
stats_df = stats_df[['screen_name','dummy_count','retweet_count',
                                 'tweet_sentiment_polarity','tweet_sentiment_subjectivity']]
stats_df = stats_df.rename(columns={"retweet_count":"total_retweet_count","dummy_count":"total_tweet_count",
                                                "tweet_sentiment_polarity":"agg_sentiment_polarity",
                                                "tweet_sentiment_subjectivity":"agg_sentiment_subjectivity"})
stats_df = stats_df[stats_df['total_retweet_count']>=retweet_threshold]
stats_df['agg_sentiment_polarity'] = stats_df['agg_sentiment_polarity']/stats_df['total_tweet_count']
stats_df['agg_sentiment_subjectivity'] = stats_df['agg_sentiment_subjectivity']/stats_df['total_tweet_count']
stats_df.reset_index(drop=True,inplace=True)
print(stats_df['agg_sentiment_polarity'].max())
print(stats_df['agg_sentiment_polarity'].min())
print(stats_df.shape)
stats_df.tail()

0.7635416666666666
-0.5
(32, 5)


,screen_name,total_tweet_count,total_retweet_count,agg_sentiment_polarity,agg_sentiment_subjectivity
27,love4thegameAK,1,126,-0.00500,0.395000
28,marceldy,1,12,0.28125,0.500000
29,mgdowney,1,14,0.00000,0.066667
30,outro_hoseok,1,108,0.00000,0.000000
31,shayxbee,1,13,0.00000,0.000000


### checking the number of people tweeted about the hashtag this week

In [192]:
tweeters = set(tweet_df['screen_name'])
all_tweeters = tweeters
print("Number of #lebanonprotest in 1 week  is: " + str(len(all_tweeters)))



Number of #lebanonprotest in 1 week  is: 273


#### Screen only the most popular twitter users whose aggregate retweets for the week > retweet_threshold

In [124]:
popular_tweeters = set(stats_df['screen_name'])
print("Number of #lebanonprotest (popular_tweeters + retweet_threshold) tweeters in 1 week is: " + str(len(popular_tweeters)))



Number of #lebanonprotest (popular_tweeters + retweet_threshold) tweeters in 1 week is: 32


#### finding edges of the network

In [201]:
nodes = list(popular_tweeters)
def create_pairings(source):
        result = []
        for p1 in range(len(source)):
                for p2 in range(p1+1,len(source)):
                        result.append([source[p1],source[p2]])
        return result

pairings = create_pairings(nodes)
print("%d pairings" % len(pairings))

#Map the twitter user names into ids

tweeters = sorted(list(set(list(tweet_df['screen_name']))))
tweeters_id = [('user_'+str(i)) for i in range(len(tweeters))]
mapping_dict = {tweeters[i]:tweeters_id[i] for i in range(len(tweeters))}
tweet_df['screen_name'] = tweet_df['screen_name'].map(mapping_dict)

#---#
screen_name_cols = ['source_screen_name','destination_screen_name']
network_df = pd.DataFrame(pairings, columns = screen_name_cols)
network_df['has_mutual_following'] = False #Initialize it to false then compute the follower friend mutual relations
network_df['source_follow_dest'] = False
network_df['dest_follow_source'] = False  #source is a friend of dest
print(network_df.shape)
network_df.head()

496 pairings
(496, 5)


,source_screen_name,destination_screen_name,has_mutual_following,source_follow_dest,dest_follow_source
0,Cairo67Unedited,MagCrisis,False,False,False
1,Cairo67Unedited,_lailaa1,False,False,False
2,Cairo67Unedited,ahmad_safadi1,False,False,False
3,Cairo67Unedited,IranFredomWatch,False,False,False
4,Cairo67Unedited,PressTV,False,False,False


#### Mutual following info among the list of popular the users "takes about 1 hour"

In [164]:
nodes = list(popular_tweeters)
network_df.to_csv('100000_Tweets.csv'+str(retweet_threshold))

def create_pairings(source):
        result = []
        for p1 in range(len(source)):
                for p2 in range(p1+1,len(source)):
                        result.append([source[p1],source[p2]])
        return result

pairings = create_pairings(nodes)
print("%d pairings" % len(pairings))

screen_name_cols = ['source_screen_name','destination_screen_name']
network_df = pd.DataFrame(pairings, columns = screen_name_cols)
network_df['has_mutual_following'] = False #Initialize it to false then compute the follower friend mutual relations
network_df['source_follow_dest'] = False
network_df['dest_follow_source'] = False  #source is a friend of dest



print(network_df.shape)
network_df.head()


496 pairings
(496, 5)


,source_screen_name,destination_screen_name,has_mutual_following,source_follow_dest,dest_follow_source
0,Cairo67Unedited,MagCrisis,False,False,False
1,Cairo67Unedited,_lailaa1,False,False,False
2,Cairo67Unedited,ahmad_safadi1,False,False,False
3,Cairo67Unedited,IranFredomWatch,False,False,False
4,Cairo67Unedited,PressTV,False,False,False


In [167]:
network_df = pd.read_csv('100000_Tweets.csv'+str(retweet_threshold))#load from saved
network_df = network_df.drop(columns='Unnamed: 0',axis=1)
network_df['source_screen_name'] = network_df['source_screen_name'].map(mapping_dict)
network_df['destination_screen_name'] = network_df['destination_screen_name'].map(mapping_dict)
print(network_df.shape)
network_df.head()

(496, 5)


,source_screen_name,destination_screen_name,has_mutual_following,source_follow_dest,dest_follow_source
0,NaN,NaN,False,False,False
1,NaN,NaN,False,False,False
2,NaN,NaN,False,False,False
3,NaN,NaN,False,False,False
4,NaN,NaN,False,False,False


#### Number of egdes (based on mutual following in the network)

In [126]:
network_df['has_mutual_following'].sum()

7

In [152]:
network_df.at[index,'source_follow_dest'] = (ff_rel[0].following == True)
network_df.at[index,'dest_follow_source'] = (ff_rel[0].followed_by == True)


network_df['source_follow_dest'].sum() + network_df['dest_follow_source'].sum()

199998

In [198]:
network_stats_df = stats_df[stats_df['screen_name'].isin(all_tweeters)].copy()
network_stats_df.reset_index(drop=True,inplace=True)

print(network_stats_df.shape)
network_stats_df.head()

(0, 5)


,screen_name,total_tweet_count,total_retweet_count,agg_sentiment_polarity,agg_sentiment_subjectivity


### Plot the mutual relation using networkx library

In [197]:
network_stats_df = stats_df[stats_df['screen_name'].isin(all_tweeters)].copy()

nodes = list(network_stats_df['screen_name'])
pd.DataFrame(nodes,columns=['Twitter_users']).to_csv('nodes_list_retweet_thresh_'+str(retweet_threshold)+'.csv')
size_of_nodes = list(network_stats_df['total_retweet_count'])
color_of_nodes = list(network_stats_df['agg_sentiment_polarity']*2) #Multiply by 2 to see more contrast in colors
mutual_follow_lol = network_df[['source_screen_name','destination_screen_name']][network_df['has_mutual_following']==True].values.tolist()
pd.DataFrame(mutual_follow_lol,columns=['Twitter_user_1','Twitter_user_2'])
mutual_follow_edges = []
for mfe in mutual_follow_lol:
    mutual_follow_edges.append((mfe[0],mfe[1]))

G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(mutual_follow_edges)

#### Graph Algorithms : Connectivity

In [211]:
### takes time
import networkx as nx 

from networkx.algorithms import approximation as approx
network_local_node_connectivity = approx.local_node_connectivity(G.degree(Cairo67Unedited),G.degree(MagCrisis)
network_local_node_connectivity(G)



SyntaxError: invalid syntax (<ipython-input-211-8ca4573dd7b1>, line 6)

In [212]:
import pandas as ps

tweets = ps.read_csv("10_Tweets.csv", memory_map = True)